In [1]:
! pip -q install pandas plotly
! pip -q install nbformat

In [158]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import glob
import json
from plotly.subplots import make_subplots
import math

In [159]:
result_files = glob.glob("/scratch/project_462000353/amanda/register-training/register-model-training/eval_results/multiple/*/global_step*/*")
print(result_files)

['eval_results/multiple/dtp/global_step5000/results_2024-12-12T21-03-54.434108.json', 'eval_results/multiple/dtp/global_step6000/results_2024-12-12T21-19-49.686628.json', 'eval_results/multiple/dtp/global_step2000/results_2024-12-12T20-18-02.706540.json', 'eval_results/multiple/dtp/global_step3000/results_2024-12-12T20-33-18.330995.json', 'eval_results/multiple/dtp/global_step11000/results_2024-12-12T22-36-54.754138.json', 'eval_results/multiple/dtp/global_step13000/results_2024-12-12T23-07-27.116325.json', 'eval_results/multiple/dtp/global_step1000/results_2024-12-12T20-02-45.250585.json', 'eval_results/multiple/dtp/global_step9000/results_2024-12-12T22-05-42.820256.json', 'eval_results/multiple/dtp/global_step12000/results_2024-12-12T22-52-11.313886.json', 'eval_results/multiple/dtp/global_step10000/results_2024-12-12T22-21-03.081917.json', 'eval_results/multiple/dtp/global_step7000/results_2024-12-12T21-35-07.587617.json', 'eval_results/multiple/dtp/global_step14000/results_2024-12-

In [161]:
millnames = ['','k','M','B','T']

def millify(n):
    n = float(n)
    millidx = max(0,min(len(millnames)-1,
                        int(math.floor(0 if n == 0 else math.log10(abs(n))/3))))

    return '{:.0f}{}'.format(n / 10**(3 * millidx), millnames[millidx])


data = []
for f in result_files:
    register = f.split("/")[2]
    step = f.split("/")[3].replace("global_step", "")
    with open(f, 'r') as result:
        j = json.load(result)
        results_all = j["results"]["all"]
        results_hellaswag_0 = {k+"_hellaswag":v for k,v in j["results"]["leaderboard|hellaswag|0"].items()}
        results_arc_easy_0 = {k+"_arc_easy":v for k,v in j["results"]["lighteval|arc:easy|0"].items()}
        results_openbookqa_0 = {k+"_openbookqa":v for k,v in j["results"]["lighteval|openbookqa|0"].items()}
        results_piqa_0 = {k+"_piqa":v for k,v in j["results"]["lighteval|piqa|0"].items()}
        df = pd.DataFrame.from_dict({register:{"step":int(step), **results_all, **results_hellaswag_0, **results_arc_easy_0, **results_openbookqa_0, **results_piqa_0}})
        #display(df)
        data.append(df.T)
        #print(register, j["results"]["all"])
        #data[register] = {step: j["results"]["all"]}

df = pd.concat(data)
df = df.reset_index()
df = df.rename(columns={"index":"register"})
# problem: step does not go to correct place on the plot
# sort it as numerical, then make it categorical so plotting works
# ordered=True does nothing here because plotly does not respect it; only sorting works
df=df.sort_values("step")
df["tokens"] = df["step"]*2.1e6
#df["step"] = pd.Categorical(df['step'], categories=sorted(df['step'].astype(int).unique()), ordered=True)
df["step"]= df['step'].apply(lambda x: str(round(x)))
df["tokens"]= df['tokens'].apply(lambda x: millify(x))
display(df)

,register,acc,acc_arc_easy,acc_hellaswag,acc_norm,acc_norm_arc_easy,acc_norm_hellaswag,acc_norm_openbookqa,acc_norm_piqa,acc_norm_stderr,...,acc_norm_stderr_piqa,acc_openbookqa,acc_piqa,acc_stderr,acc_stderr_arc_easy,acc_stderr_hellaswag,acc_stderr_openbookqa,acc_stderr_piqa,step,tokens
104,IP,0.303884,0.270623,0.259211,0.322348,0.263889,0.255726,0.254,0.515778,0.011136,...,0.011660,0.140,0.545702,0.010160,0.009116,0.004373,0.015533,0.011617,1000,2B
34,HI,0.311792,0.269781,0.260307,0.338167,0.268098,0.254033,0.256,0.574538,0.011127,...,0.011535,0.142,0.575082,0.010161,0.009108,0.004379,0.015626,0.011534,1000,2B
48,NA,0.299615,0.260943,0.259610,0.328137,0.266414,0.251444,0.280,0.514690,0.011290,...,0.011661,0.142,0.535909,0.010162,0.009011,0.004375,0.015626,0.011636,1000,2B
6,dtp,0.307218,0.284091,0.258713,0.326932,0.281987,0.256124,0.262,0.507617,0.011235,...,0.011664,0.142,0.544070,0.010218,0.009254,0.004370,0.015626,0.011620,1000,2B
20,ne,0.297054,0.250842,0.259112,0.326545,0.263047,0.244473,0.272,0.526659,0.011223,...,0.011649,0.138,0.540261,0.010084,0.008895,0.004373,0.015440,0.011628,1000,2B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53,NA,0.325416,0.326178,0.268771,0.343246,0.313131,0.269667,0.220,0.570185,0.011010,...,0.011550,0.130,0.576714,0.010157,0.009620,0.004424,0.015055,0.011528,14000,29B
25,ne,0.317650,0.325758,0.264788,0.341378,0.320707,0.265983,0.244,0.534820,0.011213,...,0.011638,0.130,0.550054,0.010171,0.009617,0.004403,0.015055,0.011607,14000,29B
109,IP,0.316223,0.292508,0.268771,0.345666,0.292929,0.277933,0.234,0.577802,0.011071,...,0.011524,0.122,0.581610,0.009980,0.009335,0.004424,0.014651,0.011509,14000,29B
95,OP,0.333752,0.327862,0.269568,0.353782,0.316498,0.278829,0.242,0.577802,0.011179,...,0.011524,0.144,0.593580,0.010309,0.009633,0.004428,0.015717,0.011460,14000,29B


In [175]:
filter_ = df["step"].isin(["3000","7000","10000","14000"])
fig = px.bar(df[filter_], x="register", y="acc_norm", error_y="acc_norm_stderr", barmode='group', color="step")
fig.update_layout(
    title="Average accuracy by Register",
    height=500,  # Adjust the height of the plot
    width=1300,
    showlegend=True,  # Show legend
)
fig.show()


In [176]:
fig = px.line(df, y="acc_norm", x="tokens", color="register")
fig.update_layout(
    title="Average accuracy by Register",
    height=600,  # Adjust the height of the plot
    width=1100,
    showlegend=True,  # Show legend
)
fig.show()

In [168]:
unique_registers = df["register"].unique()
#colors = px.colors.sample_colorscale("mygbm",len(unique_registers))  # for other colormaps
c_map={}
for i, register in enumerate(unique_registers):
    c_map[register] = px.colors.qualitative.Plotly[i]   # this is the same as above
print(c_map)



{'IP': '#636EFA', 'HI': '#EF553B', 'NA': '#00CC96', 'dtp': '#AB63FA', 'ne': '#FFA15A', 'OP': '#19D3F3', 'IN': '#FF6692', 'ID': '#B6E880'}


In [170]:


# Create a 2x2 grid of subplots
fig = make_subplots(
    rows=2, cols=2, 
    subplot_titles=["arc:easy", "hellaswag", "piqa", "openbookqa"],
    x_title="Tokens",
    y_title="Accuracy score",
    #shared_xaxes=True,  # Share the x-axis across all subplots
    vertical_spacing=0.15,  # Adjust space between subplots
)

# List of accuracy columns
accuracy_columns = ['acc_norm_arc_easy', 'acc_norm_hellaswag', 'acc_norm_piqa', 'acc_norm_openbookqa']

# Loop over each accuracy metric and add traces to the subplots
for i, accuracy_col in enumerate(accuracy_columns):
    row = (i // 2) + 1  # Determine the row for the subplot (1 or 2)
    col = (i % 2) + 1   # Determine the column for the subplot (1 or 2)
    
    # Group by label and plot each group with a different color
    for label in df['register'].unique():
        label_data = df[df['register'] == label]
        
        fig.add_trace(
            go.Scatter(
                x=label_data['tokens'], 
                y=label_data[accuracy_col], 
                mode='lines', 
                showlegend=True if row==1 and col==1 else False,
                name=f'{label}', #if row==0 and col==0 else None,#name=f'{accuracy_col} - {label}', 
                legendgroup=f'{label}',
                line=dict(color=c_map[label])#dict(color=px.colors.qualitative.Set1[list(df['register'].unique()).index(label)])
            ),
            row=row, col=col
        )

# Update layout for better appearance
fig.update_layout(
    title="Accuracy Metrics by Task and Register",
    height=800,  # Adjust the height of the plot
    width=1300,
    showlegend=True,  # Show legend
)


# Show the plot
fig.show()
